# Acquire the seed words for guided LDA/Bertopic

In [1]:
import pandas as pd
import os
import ast
import advertools as adv
import re
from collections import Counter, defaultdict

## Load CSV as data

In [2]:
data = pd.read_csv("sectioned_data.csv")
print(len(data))
data

100


,Unnamed: 0,ecli,date,inhoudsindicatie,fulltext,sections
0,0,ECLI:NL:CBB:2022:1,2022-01-11,"Artikel 2:3, eerste lid, van de Algemene wet ...",uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJ...,['\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLL...
1,1,ECLI:NL:CBB:2022:10,2022-01-18,Mond - en klauwzeer. Medio maart 2001 is in N...,Uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJ...,['\n\nUitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLL...
2,2,ECLI:NL:CBB:2022:100,2022-03-08,-\tBeleidsregel tegemoetkoming ondernemers ge...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLE...
3,3,ECLI:NL:CBB:2022:101,2022-03-08,"Warenwet, hoger beroep, Verordening 853/2004,...",uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLE...
4,4,ECLI:NL:CBB:2022:102,2022-03-08,Regeling garanties van oorsprong en certifica...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLE...
...,...,...,...,...,...,...
95,95,ECLI:NL:CBB:2022:185,2022-04-20,"Beslissing wrakingskamer. Artikel 3, lid 4, a...",beslissing COLLEGE VAN BEROEP VOOR HET BEDRI...,['\n\nbeslissing \n\n\n\n\n\n\n\n\n\n\n\n\nCOL...
96,96,ECLI:NL:CBB:2022:186,2022-04-26,-\tBeleidsregel Regeling subsidie vaste laste...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLE...
97,97,ECLI:NL:CBB:2022:187,2022-04-26,Last onder bestuursdwang en kostenbesluit. Ve...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLE...
98,98,ECLI:NL:CBB:2022:188,2022-04-26,GLB. Uitbetaling van de basis- en vergroening...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,['\n\nuitspraak\n\n\n\n\n\nCOLLEGE VAN BEROEP ...


In [4]:
data = data.dropna(subset=['fulltext'])
#data['sections'] = data['sections'].apply(lambda list_of_strings: [re.sub('\n+', ' ', str(s)) for s in  ast.literal_eval(list_of_strings)])
data['sections'] = data['sections'].apply(lambda list_of_strings: [
    re.sub('[^\w\s\[\]]', '',  # Replace anything that is not a word character, whitespace, or square bracket with nothing
    re.sub('[0-9]+', '',  # Remove numbers
    re.sub('\n+', ' ', s.lower())))  # Replace newline characters with a space and convert to lowercase
    for s in ast.literal_eval(list_of_strings)  # Safely evaluate the string as a list
])
print(len(data))
data

97


C:\Users\Chloe\AppData\Local\Temp\ipykernel_30364\3481923288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sections'] = data['sections'].apply(lambda list_of_strings: [


,Unnamed: 0,ecli,date,inhoudsindicatie,fulltext,sections
0,0,ECLI:NL:CBB:2022:1,2022-01-11,"Artikel 2:3, eerste lid, van de Algemene wet ...",uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJ...,[ uitspraak college van beroep voor het bedri...
1,1,ECLI:NL:CBB:2022:10,2022-01-18,Mond - en klauwzeer. Medio maart 2001 is in N...,Uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJ...,[ uitspraak college van beroep voor het bedri...
2,2,ECLI:NL:CBB:2022:100,2022-03-08,-\tBeleidsregel tegemoetkoming ondernemers ge...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...
3,3,ECLI:NL:CBB:2022:101,2022-03-08,"Warenwet, hoger beroep, Verordening 853/2004,...",uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...
4,4,ECLI:NL:CBB:2022:102,2022-03-08,Regeling garanties van oorsprong en certifica...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...
...,...,...,...,...,...,...
95,95,ECLI:NL:CBB:2022:185,2022-04-20,"Beslissing wrakingskamer. Artikel 3, lid 4, a...",beslissing COLLEGE VAN BEROEP VOOR HET BEDRI...,[ beslissing college van beroep voor het bedr...
96,96,ECLI:NL:CBB:2022:186,2022-04-26,-\tBeleidsregel Regeling subsidie vaste laste...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...
97,97,ECLI:NL:CBB:2022:187,2022-04-26,Last onder bestuursdwang en kostenbesluit. Ve...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...
98,98,ECLI:NL:CBB:2022:188,2022-04-26,GLB. Uitbetaling van de basis- en vergroening...,uitspraak COLLEGE VAN BEROEP VOOR HET BEDRIJF...,[ uitspraak college van beroep voor het bedrij...


In [ ]:
print(data.sections[0])
print(len(data.sections[0]))

In [ ]:
# Only get common words for each section without taking into account duplicate words and which section has each word most commonly

def get_common_words_per_section(section_texts, stopwords_dutch, num_common=10):
    # Initialize an empty list to hold the common words from each section
    common_words_per_section = []
    # Iterate over each section text in the list
    for section_text in section_texts:
        # Split the section text into words, filter out stopwords, and count the words
        word_counts = Counter([word for word in section_text.split() if word not in stopwords_dutch])
        # Get the num_common most common words from this section
        common_words = [word for word, _ in word_counts.most_common(num_common)]
        # Add this section's common words to the list
        common_words_per_section.append(common_words)
    # Return the list of common words for each section
    return common_words_per_section

In [5]:
# For each section get unique most common words where the section of some very common words is based on frequency of occurrence

def get_unique_common_words(section_texts, stopwords_dutch, num_common=10):
    section_word_counts = []
    overall_word_counts = Counter()

    # Step 1 & 2: Identify common words per section and track overall frequencies
    for section_text in section_texts:
        word_counts = Counter([word for word in section_text.split() if word not in stopwords_dutch])
        section_word_counts.append(word_counts)
        overall_word_counts += word_counts

    # Step 3: Determine the most representative section for each word
    word_to_section = defaultdict(list)
    for i, word_counts in enumerate(section_word_counts):
        for word in word_counts:
            word_to_section[word].append((word_counts[word], i))  # (frequency, section_index)

    # Choose the section where the word is most frequent
    most_representative_section = {word: max(sections, key=lambda x: x[0])[1] for word, sections in word_to_section.items()}

    # Step 4: Compile unique common words for each section
    unique_common_words_per_section = []
    for i, word_counts in enumerate(section_word_counts):
        common_words = [word for word, _ in word_counts.most_common(num_common)]
        unique_common_words = [word for word in common_words if most_representative_section[word] == i]
        unique_common_words_per_section.append(unique_common_words)

    return unique_common_words_per_section

In [ ]:
# add a column for common words or unique common words
stopwords_dutch = list(adv.stopwords['dutch'])
#data['common_words'] = data['sections'].apply(lambda x: get_common_words_per_section(x, stopwords_dutch))
data['unique_common_words'] = data['sections'].apply(lambda x: get_unique_common_words(x, stopwords_dutch))

In [ ]:
# print common words
stopwords_dutch = list(adv.stopwords['dutch'])
for index, row in data.iterrows():
    common_words_list = get_common_words_per_section(row['sections'], stopwords_dutch)
    print(f"Section {index + 1}: {common_words_list}")

In [6]:
# print unique common words
stopwords_dutch = list(adv.stopwords['dutch'])
for index, row in data.iterrows():
    unique_common_words_list = get_unique_common_words(row['sections'], stopwords_dutch)
    print(f"Section {index + 1}: {unique_common_words_list}")

Section 1: [['bedrijfsleven', 'zaaknummer', 'meervoudige', 'kamer', 'januari', 'zaak', 'tussen'], ['besluit', 'oktober', 'primaire', 'onderzoek', 'nadere', 'brief'], ['overwegingen'], ['augustus', 'vloer'], ['navolging', 'emailbericht', 'datum', 'ingebruikname'], ['appellant', 'aanvraag', 'verweerder', 'warmtepomp', 'isde', 'seeh', 'subsidie', 'eerste', 'lid', 'artikel'], ['uitspraak', 'mr', 'beslissing', 'draagt', 'griffier', 'verklaart']]
Section 2: [['mr', 'gemachtigden', 'minister', 'ml', 'batting', 'bedrijfsleven'], ['appellant', 'dieren', 'verklaard', 'beroep', 'onder'], ['besluit', 'verweerder', 'uitspraak', 'bestreden', 'primaire', 'bezwaar'], ['kalf', '[naam', ']', 'monster', 'nfirapport', 'heparinemonster', 'dr', 'juni'], ['uitgegaan', 'beroepsprocedure', 'onverenigbaar', 'rechtmatigheid', 'onrechtmatigheid'], ['bedrijf', 'monsters', 'rvv'], ['maatregelen', 'binnen', 'volgens', 'alle', 'bedrijven', 'straal', 'besmet'], ['verzoekt', 'tenslotte', 'vanwege'], ['testuitslag', 'po

In [ ]:
data